In [3]:
import matplotlib
from matplotlib import style
style.use('seaborn')
import matplotlib.pyplot as plt

import pandas as pd

# Import SQLAlchemy `automap` and other dependencies here
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func
 

In [4]:
from config import connection_string

engine = create_engine(connection_string)
connection = engine.connect()


In [6]:
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

['title', 'employee', 'department', 'salary']

In [5]:
inspector = inspect(engine)
inspector.get_table_names()

['title', 'employee', 'dept_emp', 'department', 'dept_manager', 'salary']

In [8]:
session = Session(bind=engine)

In [13]:
Employee = Base.classes.employee
Salary = Base.classes.salary
Title = Base.classes.title

In [40]:

results =  pd.read_sql(session.query(Title.title, func.avg(Salary.salary).label('average'))\
                    .join(Employee, Employee.emp_title_id == Title.title_id)\
                    .join(Salary, Salary.emp_no == Employee.emp_no).group_by(Title.title).statement, engine)

results.head()

colors = []

import matplotlib._color_data as mcd
# build colors 

overlap = {name for name in mcd.CSS4_COLORS
           if "xkcd:" + name in mcd.XKCD_COLORS}
i = 0
for j, n in enumerate(sorted(overlap, reverse=True)):
    weight = None
    cn = mcd.CSS4_COLORS[n]
    colors.append(cn)
    if (i >= results['title'].count()):
        break;
    else
       i 
for i in range(1, results['title'].count()) :
    colors.append( mcd.CSS4_COLORS[i])


results["Color"] = colors
#Set Index to title - ONLY RUN ONCE
results = results.set_index("title")


KeyError: 1

In [46]:

import matplotlib._color_data as mcd
# build colors 

overlap = {name for name in mcd.CSS4_COLORS
           if "xkcd:" + name in mcd.XKCD_COLORS}

for j, n in enumerate(sorted(overlap, reverse=True)):
    weight = None
    cn = mcd.CSS4_COLORS[n]
    print (cn)


#9ACD32
#FFFF00
#FFFFFF
#F5DEB3
#EE82EE
#40E0D0
#FF6347
#008080
#D2B48C
#C0C0C0
#A0522D
#FA8072
#FF0000
#800080
#DDA0DD
#FFC0CB
#DA70D6
#FF4500
#FFA500
#808000
#000080
#800000
#FF00FF
#00FF00
#90EE90
#ADD8E6
#E6E6FA
#F0E68C
#FFFFF0
#4B0082
#808080
#008000
#DAA520
#FFD700
#FF00FF
#006400
#00008B
#00FFFF
#DC143C
#FF7F50
#D2691E
#7FFF00
#A52A2A
#0000FF
#000000
#F5F5DC
#F0FFFF
#7FFFD4
#00FFFF


In [34]:

plt.figure(figsize=(25,15))
ax = results["average"].plot(kind="bar", color=results["Color"], alpha=.8,align="center",rot=0)
ylabels = ['{:,.2f}'.format(x) for x in ax.get_yticks()]
ax.set_yticklabels(ylabels, fontsize=23)

for rect in ax.patches:
    # Find where everything is located
    height = rect.get_height()
    width = rect.get_width()
    x = rect.get_x()
    y = rect.get_y()
    # The width of the bar is the data value and can used as the label
    label_text = f"{round(height,2)}"  # f'{height:.2f}' if you have decimal values as labels
    label_x = x + width / 2
    label_y = y + height + 1
    ax.text(label_x, label_y, label_text, ha='center', va='center', fontsize=20)

plt.xlabel("Titles",fontsize=30)
plt.xticks(fontsize=20)
plt.ylabel("Average Salary",fontsize=30)
plt.title("Average Salary per Title", fontsize=35)
plt.tight_layout()
plt.show()


NameError: name 'dom_df' is not defined

<Figure size 1800x1080 with 0 Axes>